In [6]:
import pandas as pd
import numpy as np
import torch

In [7]:
gam_df = pd.read_csv('gam.csv')
gyul_df = pd.read_csv('gyul.csv')

In [8]:
def get_data(df):
    df = df[df['item'] == 'TG']
    df = df[df['corporation'] == 'A']
    df = df[df['location'] == 'J']
    df = df[df['price(원/kg)'] > 0]
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    return df

df = get_data(gam_df)

In [9]:
input_df = df[['month', 'day', 'supply(kg)']]
label_df = df[['price(원/kg)']]

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
input_scaled = scaler.fit_transform(input_df)
input_scaled = torch.Tensor(input_scaled).float()
label_scaling = 1000
target = torch.Tensor(label_df.to_numpy()).float() / label_scaling

In [11]:
num_test = 100
x_train = input_scaled[:-num_test,:]
y_train = target[:-num_test]
x_test = input_scaled[-num_test:,:]
y_test = target[-num_test:]

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
from torch.utils.data import Dataset, DataLoader
input_size = 3
hidden_size = 128
batch_size = 16

# 데이터셋 클래스 정의
class PriceDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# 데이터셋과 데이터로더 생성
train_dataset = PriceDataset(x_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = PriceDataset(x_test, y_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [14]:
class CustomModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CustomModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, hidden = self.rnn(x)
        out = self.linear(out)
        return out

learning_rate = 0.005
model = CustomModel(input_size, hidden_size, 1)

In [15]:
from torch.optim import Adam

def train_model(model, train_loader, epochs):
    model.train()
    criterion = nn.MSELoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        total_loss = 0
        for i, (inputs, prices) in enumerate(train_loader):
            outputs = model(inputs.cuda())
            loss = criterion(outputs, prices.cuda())
            total_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}')
        
def test_model(model, test_loader):
    model.eval()
    criterion = nn.L1Loss()

    # 테스트 과정
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.cuda())
            loss = criterion(outputs, labels.cuda())
            total_loss += loss.item()

    average_loss = total_loss / len(test_dataset) * label_scaling
    print(f'Test Loss: {average_loss:.4f}')

In [16]:
model.cuda()
train_model(model, train_loader, 300)

Epoch [1/300], Loss: 9.3039
Epoch [2/300], Loss: 8.5726
Epoch [3/300], Loss: 8.6175
Epoch [4/300], Loss: 8.2966
Epoch [5/300], Loss: 8.3447
Epoch [6/300], Loss: 8.2431
Epoch [7/300], Loss: 8.1876
Epoch [8/300], Loss: 7.9480
Epoch [9/300], Loss: 8.1659
Epoch [10/300], Loss: 7.8819
Epoch [11/300], Loss: 7.7638
Epoch [12/300], Loss: 7.5778
Epoch [13/300], Loss: 7.1581
Epoch [14/300], Loss: 6.9147
Epoch [15/300], Loss: 6.7362
Epoch [16/300], Loss: 6.4904
Epoch [17/300], Loss: 5.9126
Epoch [18/300], Loss: 5.6860
Epoch [19/300], Loss: 5.7985
Epoch [20/300], Loss: 5.2308
Epoch [21/300], Loss: 5.2502
Epoch [22/300], Loss: 5.2788
Epoch [23/300], Loss: 4.7330
Epoch [24/300], Loss: 4.6696
Epoch [25/300], Loss: 4.5301
Epoch [26/300], Loss: 4.4160
Epoch [27/300], Loss: 4.9512
Epoch [28/300], Loss: 4.6233
Epoch [29/300], Loss: 4.6194
Epoch [30/300], Loss: 5.2313
Epoch [31/300], Loss: 4.8027
Epoch [32/300], Loss: 4.0618
Epoch [33/300], Loss: 3.9817
Epoch [34/300], Loss: 3.7759
Epoch [35/300], Loss: 3

In [17]:
test_model(model, test_loader)

Test Loss: 40.3644
